## 构建股票信息字典
目标：构建全量的A股代码列表，标准化股票代码，获取全量的名称信息（含 股票简称、公司名称、历史简称、历史全称等）以便用来关联新闻资讯中提到的公司名称，并附加企业基础信息。  
注意：  
· 证券简称、上市日期、总股本、流通股本 字段对应A、B股各自的市场板块，股本的单位是 股。

## 股票简称业务涵义
· ST，亏损股。对连续两个会计年度都出现亏损的公司施行的特别处理。  
· *ST，是指连续三年亏损，有退市风险的股票。  
· SST，指还没有进行股改的连续两个会计年度都出现亏损的公司。  
· S*ST，指公司经营连续三年亏损，进行退市预警和还没有完成股改。  

In [1]:
#! /usr/bin/env python3
# -*- coding: utf-8 -*-

import akshare as ak
import pandas as pd
import random
import time
import os

# 从配置文件中读取密钥，实现配置和代码分离。参考： https://juejin.cn/post/7099283807953977358
from dotenv import dotenv_values, load_dotenv
load_dotenv('.env') 
env_vars = dotenv_values()

# 使用代理，使用快代理的隧道，用户名密码方式 https://www.kuaidaili.com/tps 。也可尝试 https://www.kuaidaili.com/free/inha/ 的免费代理。
# 可以通过修改环境变量来设置Python requests库的代理。AKShare库使用的是requests库，requests库会自动读取环境变量HTTP_PROXY和HTTPS_PROXY来设置HTTP和HTTPS的代理。
username = env_vars['kdl_username']
password = env_vars['kdl_password']
tunnel = env_vars['kdl_tunnel']
os.environ['http_proxy'] = "http://%(user)s:%(pwd)s@%(proxy)s/" % {"user": username, "pwd": password, "proxy": tunnel}

In [2]:
stock_info_df = []  # 股票信息字典初始化

# 获取股票列表-上证
stock_info_sh_a_df = ak.stock_info_sh_name_code(symbol="主板A股").rename(columns={'公司全称': '证券全称','证券代码':'A股代码'})
stock_info_sh_b_df = ak.stock_info_sh_name_code(symbol="主板B股").rename(columns={'公司全称': '证券全称','证券代码':'B股代码'})
stock_info_sh_kc_df = ak.stock_info_sh_name_code(symbol="科创板").rename(columns={'公司全称': '证券全称','证券代码':'A股代码'})
# 添加一列，标识板块
stock_info_sh_a_df['板块'] = '主板A股'
stock_info_sh_b_df['板块'] = '主板B股'
stock_info_sh_kc_df['板块'] = '科创板'
# 将以上上证df合并
stock_info_sh_df = pd.concat([stock_info_sh_a_df, stock_info_sh_b_df, stock_info_sh_kc_df], axis=0) # 上证股票列表
# 添加一列，标识交易所
stock_info_sh_df['交易所'] = '上交所'
# 添加一列，标识证券完整代码。优先取A股代码，如果A股代码为空，则取B股代码。  

stock_info_sh_df.to_csv('data/stock_info_sh.csv', encoding='utf-8-sig') # 保存到本地

# 获取股票列表-深证，重命名列名
stock_info_sz_a_df = ak.stock_info_sz_name_code(symbol="A股列表").rename(columns={'A股简称': '证券简称','A股上市日期':'上市日期','A股总股本':'总股本','A股流通股本':'流通股本'})   # A股列表
stock_info_sz_b_df = ak.stock_info_sz_name_code(symbol="B股列表").rename(columns={'B股简称': '证券简称','B股上市日期':'上市日期','B股总股本':'总股本','B股流通股本':'流通股本'})   # B股列表
stock_info_sz_ab_df = ak.stock_info_sz_name_code(symbol="AB股列表").rename(columns={'B股简称': '证券简称','B股上市日期':'上市日期'})  # AB股列表，都包含在A股列表中，但不包含在B股列表中，可以认为是B股的补充。
stock_info_sz_ab_df.drop(['A股代码','A股简称','A股上市日期'], axis=1, inplace=True)  # 移除冗余的列‘A股代码’、‘A股简称’、‘A股上市日期’

# stock_info_sz_cdr_df = ak.stock_info_sz_name_code(symbol="CDR列表")  # CDR列表为空，不处理

# 将深证df合并
stock_info_sz_df = pd.concat([stock_info_sz_a_df, stock_info_sz_b_df, stock_info_sz_ab_df], axis=0)
stock_info_sz_df['交易所'] = '深交所'    # 添加一列，标识交易所
stock_info_sz_df.to_csv('data/stock_info_sz.csv', encoding='utf-8-sig') # 保存到本地

# 获取股票列表-北证
stock_info_bj_df = ak.stock_info_bj_name_code().rename(columns={'证券代码':'A股代码'})
stock_info_bj_df['交易所'] = '北交所'   # 添加一列，标识交易所、板块
stock_info_bj_df['板块'] = '北交所'    # 添加一列，标识交易所、板块
stock_info_bj_df.to_csv('data/stock_info_bj.csv', encoding='utf-8-sig')  # 保存到本地

# 将上证、深证、北证df合并
stock_info_df = pd.concat([stock_info_sh_df, stock_info_sz_df, stock_info_bj_df], axis=0)
# 添加 '证券完整代码' 列，规则是：'证券代码'.'交易所'。'证券代码'优先从'A股代码'中选取，'A股代码'为空的，取 'B股代码'。'交易所'从'交易所'列中取值，对其中的值进行替换，规则是：'上交所'替换为'SH'，'深交所'替换为'SZ'，'北交所'替换为'BJ'。
stock_info_df['证券完整代码'] = stock_info_df['A股代码'].fillna(stock_info_df['B股代码']) + '.' + stock_info_df['交易所'].replace({'上交所': 'SH', '深交所': 'SZ', '北交所': 'BJ'})


# 对stock_info_df重新编号
stock_info_df.reset_index(drop=True, inplace=True)
# 添加空列：'H股代码', '历史简称', '历史全称', '证券英文全称'
stock_info_df['H股代码'] = ''
stock_info_df['历史简称'] = ''
stock_info_df['历史全称'] = ''
stock_info_df['证券英文全称'] = ''
# 对stock_info_df修改列名次序，并添加将要补充的列名。
stock_info_df = stock_info_df[['交易所', '板块', '证券完整代码', 'A股代码', 'B股代码', 'H股代码', '证券简称', '历史简称','证券全称','历史全称','证券英文全称', '上市日期', '总股本', '流通股本', '所属行业', '地区']]

stock_info_df.to_csv('data/stock_info.csv', encoding='utf-8-sig') # 保存到本地
# 测试结论：通过

In [3]:
# 调用 名称变更-深证 接口，一次性获取深交所的全量’历史简称‘、’历史全称‘。按'变更日期'降序。
stock_info_sz_change_name_s_df = ak.stock_info_sz_change_name(symbol="简称变更")
stock_info_sz_change_name_s_df.sort_values(by='变更日期', ascending=False, inplace=True)
stock_info_sz_change_name_f_df = ak.stock_info_sz_change_name(symbol="全称变更")
stock_info_sz_change_name_f_df.sort_values(by='变更日期', ascending=False, inplace=True)
stock_info_sz_change_name_s_df.to_csv('tmp/stock_info_sz_change_name_s.csv', encoding='utf-8-sig') # 保存到本地
stock_info_sz_change_name_f_df.to_csv('tmp/stock_info_sz_change_name_f.csv', encoding='utf-8-sig') # 保存到本地

# stock_info_df.loc[i, 'A股代码'] 去除.后面的内容，形成临时的id列，以便与stock_info_sz_change_name_s_df['证券代码']进行匹配。
stock_info_df['id'] = stock_info_df['证券完整代码'].str.split('.').str[0]

# 对stock_info_df每一行与stock_info_sz_change_name_s_df进行循环匹配，如果匹配成功，将stock_info_sz_change_name_s_df['简称变更']中的值逐一添加到stock_info_df['历史简称']中，然后对stock_info_df['历史简称']中的元素进行去重。
for i in range(len(stock_info_df)):
    for j in range(len(stock_info_sz_change_name_s_df)):
        if  stock_info_sz_change_name_s_df.loc[j, '证券代码'] == stock_info_df.loc[i,'id']: # 此处可以放心匹配。A股不同市场板块的股票代码不会重复，但是股票代码和基金、指数代码会有重复。
            stock_info_df.loc[i, '历史简称'] = stock_info_df.loc[i, '历史简称'] + ',' + stock_info_sz_change_name_s_df.loc[j, '变更后简称'] # 添加新增的历史简称。

    
# 对stock_info_df每一行与stock_info_sz_change_name_f_df进行循环匹配，如果匹配成功，将stock_info_sz_change_name_f_df['变更后全称']中的值逐一添加到stock_info_df['历史全称']中，然后对stock_info_df['历史全称']中的元素进行去重。
for i in range(len(stock_info_df)):
    for j in range(len(stock_info_sz_change_name_f_df)):  # 逐一匹配并添加'历史全称'
        if  stock_info_sz_change_name_f_df.loc[j, '证券代码'] == stock_info_df.loc[i,'id']: # 此处可以放心匹配。A股不同市场板块的股票代码不会重复，但是股票代码和基金、指数代码会有重复。
            stock_info_df.loc[i, '历史全称'] = stock_info_df.loc[i, '历史全称'] + ',' + stock_info_sz_change_name_f_df.loc[j, '变更后全称'] # 添加新增的历史简称。
    for j in range(len(stock_info_sz_change_name_f_df)):  # 使用stock_info_sz_change_name_f_df中最新的'变更前全称'，来更新stock_info_df.loc['证券全称']。
        if  stock_info_sz_change_name_f_df.loc[j, '证券代码'] == stock_info_df.loc[i,'id']: 
            stock_info_df.loc[i, '证券全称'] = stock_info_sz_change_name_f_df.loc[j, '变更前全称'] 
            break # 仅取最新值。匹配成功一次，即跳出循环。

stock_info_df.to_csv('data/stock_info.csv', encoding='utf-8-sig') # 保存到本地


# 测试结论：通过

In [4]:
# Function: 对每只股票调用 akshare.stock_profile_cninfo 接口，获取公司概况-巨潮资讯。
# 支持QPS限制；支持断点续传。
# 初始化stock_profile_cninfo，用于存储股票公司概况-巨潮资讯，'证券完整代码'列是Key。
# 如果本地文件存在，则读取本地文件；如果本地文件不存在，则初始化一个空的DataFrame
try:  # 本地文件存在
    stock_profile_cninfo = pd.read_csv('data/stock_profile_cninfo.csv')  # 每次启动，如果存在本地文件则先读取
except: # 本地文件不存在
    stock_profile_cninfo = pd.DataFrame(columns=['证券完整代码','公司名称', '英文名称', '曾用简称', 'A股代码', 'A股简称', 'B股代码', 'B股简称', 'H股代码', 'H股简称', '入选指数', '所属市场', '所属行业', '法人代表', '注册资金', '成立日期', '上市日期', '官方网站', '电子邮箱', '联系电话', '传真', '注册地址', '办公地址', '邮政编码', '主营业务', '经营范围', '机构简介'])

# 如果本地文件内容为空，也初始化列名
if len(stock_profile_cninfo) == 0:
    stock_profile_cninfo = pd.DataFrame(columns=['证券完整代码','公司名称', '英文名称', '曾用简称', 'A股代码', 'A股简称', 'B股代码', 'B股简称', 'H股代码', 'H股简称', '入选指数', '所属市场', '所属行业', '法人代表', '注册资金', '成立日期', '上市日期', '官方网站', '电子邮箱', '联系电话', '传真', '注册地址', '办公地址', '邮政编码', '主营业务', '经营范围', '机构简介']) # 初始化一个空的DataFrame

stock_info_df = pd.read_csv('data/stock_info.csv')  # 获取本地股票列表
stock_info_df['id'] = stock_info_df['证券完整代码'].str.split('.').str[0] # 新增一列'id'，为证券代码
stock_profile_cninfo_set = set(map(str, stock_profile_cninfo['证券完整代码'].str.split('.').str[0])) # 已爬取的证券完整代码集合，用于断点续传。一定要将每个元素的数据类型转为字符串，否则无法和id正常匹配。在set中查找元素的时间复杂度为O(1)，比list\array效率高。

# 对stock_info_df中的逐个股票，调用 公司概况-巨潮资讯 接口
for i in range(len(stock_info_df)):
    id = stock_info_df.loc[i, 'id']
    
    if  id not in stock_profile_cninfo_set: # 排除已经获取到stock_profile_cninfo的股票
        # 调用 公司概况-巨潮资讯 接口。如果接口调用失败，是因为有QPS限制，随机暂停n秒，然后再等待重试。
        while True: # 一直循环，直到接口调用成功。
            try: # 尝试调用接口
                profile = ak.stock_profile_cninfo(symbol=id) # 获取单条股票的公司概况-巨潮资讯
                # time.sleep(0.05) # 满足20QPS限制
                break # 如果调用成功，则跳出while循环
            except: # 如果引发了异常
                print('接口调用失败，等待0秒后重试。')
                # time.sleep(random.randint(5, 15))
        # 如果profile为空，则表示该股票没有历史简称。所以仅当profile不为空时，才将结果写入stock_profile_cninfo。
        if len(profile) != 0:
            print('已获取：'+id,profile['公司名称'])
            # 将结果带上证券完整代码，添加到stock_profile_cninfo
            profile['证券完整代码'] = stock_info_df.loc[i, '证券完整代码']
            stock_profile_cninfo = stock_profile_cninfo._append(profile, ignore_index=True) # 将结果写入stock_profile_cninfo

            # i每循环100次，将结果写入本地文件。
            if i % 100 == 0:
                stock_profile_cninfo.to_csv('data/stock_profile_cninfo.csv', index=False, encoding='utf-8-sig') # 保存为utf-8格式，避免乱码。
                print('已写入本地文件。')              
        # time.sleep(random.randint(1, 2)) # 每次调用接口后，随机暂停n秒，避免QPS限制。
stock_profile_cninfo.drop_duplicates(subset=['证券完整代码'], keep='first', inplace=True) # 去重，保留第一次出现的证券完整代码。
stock_profile_cninfo.to_csv('data/stock_profile_cninfo.csv', index=False, encoding='utf-8-sig') # 整体结束后也保存。
print('获取完毕')

# 测试结论：通过。

已获取：688716 0    吉林省中研高分子材料股份有限公司
Name: 公司名称, dtype: object
已获取：301548 0    湖南崇德科技股份有限公司
Name: 公司名称, dtype: object
获取完毕


In [5]:
# 融合已获取的 公司概况-巨潮资讯 到stock_info_df。stock_info_df 唯一有价值的列，就剩下 历史全称、总股本、流通股本，故只需要这三列。
stock_info_df= pd.read_csv('data/stock_info.csv')  # 读取本地文件
stock_info_df = stock_info_df[['证券完整代码','历史全称','总股本','流通股本']] # 仅保留有价值的列
stock_profile_cninfo_df = pd.read_csv('data/stock_profile_cninfo.csv')  # 读取公司概况-巨潮资讯

# 都进行去重，避免连接后出现重复行。
stock_profile_cninfo_df.drop_duplicates(subset=['证券完整代码'], keep='first', inplace=True) # 去重，保留第一次出现的证券完整代码。
stock_info_df.drop_duplicates(subset=['证券完整代码'], keep='first', inplace=True) # 去重，保留第一次出现的证券完整代码。

# 基于'证券完整代码'列进行融合
stock_info_df = pd.merge(stock_info_df, stock_profile_cninfo_df, how='left', on='证券完整代码')

# 保存到本地
stock_info_df.to_csv('data/stock_info.csv', encoding='utf-8-sig') # 保存到本地

In [ ]:
# 测试ak.stock_profile_cninfo函数能否支持传入多个股票代码
# stock_profile_cninfo_df = ak.stock_profile_cninfo(symbol=["600030","600031","600032","600033","600034","600035","600036"])
# print(stock_profile_cninfo_df)

# 测试结论：已经不支持传入多个股票代码。

In [7]:
# 数据清洗，大部分不用了
# stock_info_df.drop(['id'], axis=1, inplace=True) # 移除临时的id列
# 将股本转为数值型，将证券简称中的空格去掉(需要先排查NaN值)
stock_info_df['总股本'] = stock_info_df['总股本'].str.replace(',', '')
stock_info_df['流通股本'] = stock_info_df['流通股本'].str.replace(',', '')
stock_info_df['A股简称'] = stock_info_df['A股简称'].str.replace(' ', '')

# 将stock_info_df['历史简称'、'历史全称']中的元素，先去除空格，再进行去重（需要确保元素第一次出现的顺序不变，并排除NaN值）。然后将去重后的元素，重新赋值给stock_info_df['历史简称'、'历史全称']。
# 去除空格在业务上会带来一个小问题：STTCL可能引发歧义。
stock_info_df['历史全称'] = stock_info_df['历史全称'].astype(str).str.replace(' ', '').str.split(',') #去除空格，再按逗号分隔字符串
stock_info_df['曾用简称'] = stock_info_df['曾用简称'].astype(str).str.replace(' ', '').str.split(',') #去除空格，再按逗号分隔字符串
for i in range(len(stock_info_df)):
    stock_info_df.loc[i, '历史全称'] = ','.join(list(dict.fromkeys(stock_info_df.loc[i, '历史全称']))) # 去重
    stock_info_df.loc[i, '曾用简称'] = ','.join(list(dict.fromkeys(stock_info_df.loc[i, '曾用简称']))) # 去重
# 移除NaN值
stock_info_df['历史全称'] = stock_info_df['历史全称'].astype(str).str.replace('nan', '')
stock_info_df['曾用简称'] = stock_info_df['曾用简称'].astype(str).str.replace('nan', '')

stock_info_df.to_csv('data/stock_info.csv', encoding='utf-8-sig') # 保存到本地
# 测试结论：通过